**EPOCH**

In [3]:
# Epoch 5
import os
from ultralytics import YOLO
import csv

# =========================
# KONFIGURASI UTAMA
# =========================

# Path data.yaml generalisasi
DATA_YAML = r"C:/Users/USER/Documents/a skripsi/riset/data_eval_final/test generalisasi/data.yaml"

# Output folder utama
OUTPUT_DIR = "./generalisasi_eval/i10-e5"

ROOT_DIR = r"C:/Users/USER/Documents/a skripsi/riset/history/5-epoch-1020"

# Daftar model (10 path)
MODEL_PATHS = [
    os.path.join(ROOT_DIR, f"1-iter/model1/weights/best.pt"),
    os.path.join(ROOT_DIR, f"2-iter/model2/weights/best.pt"),
    os.path.join(ROOT_DIR, f"3-iter/model3/weights/best.pt"),
    os.path.join(ROOT_DIR, f"4-iter/model4/weights/best.pt"),
    os.path.join(ROOT_DIR, f"5-iter/model5/weights/best.pt"),
    os.path.join(ROOT_DIR, f"6-iter/model6/weights/best.pt"),
    os.path.join(ROOT_DIR, f"7-iter/model7/weights/best.pt"),
    os.path.join(ROOT_DIR, f"8-iter/model8/weights/best.pt"),
    os.path.join(ROOT_DIR, f"9-iter/model9/weights/best.pt"),
    os.path.join(ROOT_DIR, f"10-iter/model10/weights/best.pt"),
]

CSV_OUT = os.path.join(OUTPUT_DIR, "results.csv")

os.makedirs(OUTPUT_DIR, exist_ok=True)

# ===================== HEADER CSV =====================
header = [
    "iteration",
    "TP", "FP", "TN", "FN",
    "precision", "recall", "f1_score",
    "mAP50", "mAP50_95"
]

rows = []

# ===================== LOOP VALIDASI =====================
for idx, model_path in enumerate(MODEL_PATHS, start=1):
    print(f"\nValidating model iterasi {idx}")
    model = YOLO(model_path)

    metrics = model.val(
        data=DATA_YAML,
        imgsz=640,
        conf=0.5,
        # iou=0.5,
        project=OUTPUT_DIR,
        name=f"iterasi_{idx}",
        save_json=False,
        plots=True
    )

    # Confusion Matrix
    cm = metrics.confusion_matrix.matrix
    # Format:
    # [[TP, FP],
    #  [FN, TN(background)]]

    TP = int(cm[0, 0])
    FP = int(cm[0, 1])
    FN = int(cm[1, 0])
    TN = 0  # Tidak bermakna untuk object detection
    print(f"TP: {TP}, FP: {FP}, TN: {TN}, FN: {FN}")

    # precision = float(metrics.box.precision[0])
    # recall = float(metrics.box.recall[0])
    # f1 = 2 * precision * recall / (precision + recall + 1e-9)
    precision = float(metrics.box.p[0])
    recall = float(metrics.box.r[0])
    f1 = float(metrics.box.f1[0])


    mAP50 = float(metrics.box.map50)
    mAP5095 = float(metrics.box.map)

    rows.append([
        idx,
        TP, FP, TN, FN,
        precision, recall, f1,
        mAP50, mAP5095
    ])

# ===================== SIMPAN CSV =====================
with open(CSV_OUT, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(rows)

print("\n✅ Validation selesai")
print(f"Hasil disimpan ke: {CSV_OUT}")



Validating model iterasi 1
Ultralytics 8.3.221  Python-3.11.9 torch-2.9.0+cpu CPU (12th Gen Intel Core i5-1235U)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access  (ping: 0.20.1 ms, read: 1172.0428.2 MB/s, size: 358.5 KB)
val: Scanning C:\Users\USER\Documents\a skripsi\riset\data_eval_final\test generalisasi\test\labels.cache... 522 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 522/522 503.4Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 33/33 0.8it/s 41.7s1.3ss
                   all        522        522          1      0.971      0.985      0.816
Speed: 1.0ms preprocess, 51.1ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to C:\Users\USER\Documents\a skripsi\workv1\app\notebook_penelitian\generalisasi_eval\i10-e5\iterasi_1
TP: 507, FP: 0, TN: 0, FN: 15

Validating model iterasi 2
Ultralytics 8.3.221  Python-3.11.9 torch-2.9.0+cpu C

In [6]:
import pandas as pd
import matplotlib.pyplot as plt

# Tentukan path file CSV sesuai lokasi di komputer Anda
path_hasil = r'C:\Users\USER\Documents\a skripsi\workv1\app\notebook_penelitian\generalisasi_eval\i10-e5\results.csv'

# Membaca data dari CSV
try:
    df = pd.read_csv(path_hasil)
    print("Data berhasil dimuat.")
except FileNotFoundError:
    print(f"File tidak ditemukan di: {path_hasil}")
    # Jika menjalankan di lingkungan lokal, pastikan path sudah benar.
    # Untuk keperluan demonstrasi, kita gunakan nama file lokal jika path absolut gagal.
    df = pd.read_csv('results.csv') 

# --- Plot 1: Precision, Recall, dan F1-Score ---
plt.clf()
plt.plot(df['iteration'], df['precision'], marker='o', label='Precision', linewidth=2)
plt.plot(df['iteration'], df['recall'], marker='s', label='Recall', linewidth=2)
plt.plot(df['iteration'], df['f1_score'], marker='^', label='F1-Score', linewidth=2)

plt.title('Evaluasi Precision, Recall, dan F1-Score (5 Epoch per Iterasi)')
plt.xlabel('Iterasi')
plt.ylabel('Nilai Metrik')
plt.xticks(df['iteration'])
plt.ylim(0, 1.05)
plt.grid(True, linestyle='--', alpha=0.7)
plt.legend(loc='lower right')
# plt.savefig('metrics_5epoch_plot.png')
plt.savefig(os.path.join(OUTPUT_DIR, 'metrics_5epoch_plot.png'))
print("Grafik 'metrics_5epoch_plot.png' telah disimpan.")

# --- Plot 2: mAP50 dan mAP50-95 ---
plt.clf()
plt.plot(df['iteration'], df['mAP50'], marker='o', label='mAP50', color='red', linewidth=2)
plt.plot(df['iteration'], df['mAP50_95'], marker='s', label='mAP50-95', color='blue', linewidth=2)

plt.title('Evaluasi mAP50 dan mAP50-95 (5 Epoch per Iterasi)')
plt.xlabel('Iterasi')
plt.ylabel('Nilai mAP')
plt.xticks(df['iteration'])
plt.ylim(0, 1.05)
plt.grid(True, linestyle='--', alpha=0.7)
plt.legend(loc='lower right')
plt.savefig(os.path.join(OUTPUT_DIR, 'map_5epoch_plot.png'))
print("Grafik 'map_5epoch_plot.png' telah disimpan.")

Data berhasil dimuat.
Grafik 'metrics_5epoch_plot.png' telah disimpan.
Grafik 'map_5epoch_plot.png' telah disimpan.


In [16]:
import pandas as pd
import matplotlib.pyplot as plt

# Tentukan path file CSV
path_hasil = r'C:\Users\USER\Documents\a skripsi\workv1\app\notebook_penelitian\generalisasi_eval\i10-e5\results.csv'
FONT_SIZE = 20
OUTPUT_DIR = "./generalisasi_eval/i10-e5"
try:
    df = pd.read_csv(path_hasil)
    print("Data berhasil dimuat.")
except FileNotFoundError:
    print("File tidak ditemukan, pastikan path sudah benar.")
    # Fallback jika dijalankan di environment lain
    df = pd.read_csv('results.csv') 

# --- Pengaturan Font ---
# Mengatur font global (seperti Times New Roman jika tersedia, atau sans-serif)
plt.rcParams.update({'font.family': 'sans-serif', 'font.size': FONT_SIZE})
font_title = {'fontsize': FONT_SIZE, 'fontweight': 'bold'}
font_label = {'fontsize': FONT_SIZE}

# --- Membuat Subplot (1 Baris, 2 Kolom) ---
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Subplot Kiri: Precision, Recall, dan F1-Score
ax1.plot(df['iteration'], df['precision'], marker='o', label='Precision', linewidth=2)
ax1.plot(df['iteration'], df['recall'], marker='s', label='Recall', linewidth=2)
ax1.plot(df['iteration'], df['f1_score'], marker='^', label='F1-Score', linewidth=2)
ax1.set_title('Precision, Recall, dan F1-Score', fontdict=font_title, fontsize=FONT_SIZE)
ax1.set_xlabel('Iterasi', fontdict=font_label, fontsize=FONT_SIZE)
ax1.set_ylabel('Nilai Metrik', fontdict=font_label, fontsize=FONT_SIZE)
ax1.set_xticks(df['iteration'])
ax1.set_ylim(0, 1.05)
ax1.grid(True, linestyle='--', alpha=0.6)
ax1.legend(loc='lower right', fontsize=FONT_SIZE)

# Subplot Kanan: mAP50 dan mAP50-95
ax2.plot(df['iteration'], df['mAP50'], marker='o', label='mAP50', color='red', linewidth=2)
ax2.plot(df['iteration'], df['mAP50_95'], marker='s', label='mAP50-95', color='blue', linewidth=2)
ax2.set_title('mAP50 dan mAP50-95', fontdict=font_title, fontsize=FONT_SIZE)
ax2.set_xlabel('Iterasi', fontdict=font_label, fontsize=FONT_SIZE)
ax2.set_ylabel('Nilai mAP', fontdict=font_label, fontsize=FONT_SIZE)
ax2.set_xticks(df['iteration'])
ax2.set_ylim(0, 1.05)
ax2.grid(True, linestyle='--', alpha=0.6)
ax2.legend(loc='lower right', fontsize=FONT_SIZE)

# Menyesuaikan jarak antar subplot agar tidak tumpang tindih
plt.tight_layout()

# Simpan gambar
plt.savefig(os.path.join(OUTPUT_DIR, 'combined_evaluation_plot.png'), dpi=300)
print("Grafik gabungan 'combined_evaluation_plot.png' telah disimpan.")

Data berhasil dimuat.
Grafik gabungan 'combined_evaluation_plot.png' telah disimpan.


In [ ]:
# Epoch 10
import os
from ultralytics import YOLO
import csv

# =========================
# KONFIGURASI UTAMA
# =========================


# Path data.yaml generalisasi
DATA_YAML = r"C:/Users/USER/Documents/a skripsi/riset/data_eval_final/test generalisasi/data.yaml"

# Output folder utama
OUTPUT_DIR = "./generalisasi_eval/i10-e10"

ROOT_DIR = r"C:/Users/USER/Documents/a skripsi/riset/history/10-ep-10-it"

# Daftar model (10 path)
MODEL_PATHS = [
    os.path.join(ROOT_DIR, f"1-iter/model1/weights/best.pt"),
    os.path.join(ROOT_DIR, f"2-iter/model2/weights/best.pt"),
    os.path.join(ROOT_DIR, f"3-iter/model3/weights/best.pt"),
    os.path.join(ROOT_DIR, f"4-iter/model4/weights/best.pt"),
    os.path.join(ROOT_DIR, f"5-iter/model5/weights/best.pt"),
    os.path.join(ROOT_DIR, f"6-iter/model6/weights/best.pt"),
    os.path.join(ROOT_DIR, f"7-iter/model7/weights/best.pt"),
    os.path.join(ROOT_DIR, f"8-iter/model8/weights/best.pt"),
    os.path.join(ROOT_DIR, f"9-iter/model9/weights/best.pt"),
    os.path.join(ROOT_DIR, f"10-iter/model10/weights/best.pt"),
]

CSV_OUT = os.path.join(OUTPUT_DIR, "results.csv")

os.makedirs(OUTPUT_DIR, exist_ok=True)

# ===================== HEADER CSV =====================
header = [
    "iteration",
    "TP", "FP", "TN", "FN",
    "precision", "recall", "f1_score",
    "mAP50", "mAP50_95"
]

rows = []

# ===================== LOOP VALIDASI =====================
for idx, model_path in enumerate(MODEL_PATHS, start=1):
    print(f"\nValidating model iterasi {idx}")
    model = YOLO(model_path)

    metrics = model.val(
        data=DATA_YAML,
        imgsz=640,
        conf=0.5,
        # iou=0.5,
        project=OUTPUT_DIR,
        name=f"iterasi_{idx}",
        save_json=False,
        plots=True
    )

    # Confusion Matrix
    cm = metrics.confusion_matrix.matrix
    # Format:
    # [[TP, FP],
    #  [FN, TN(background)]]

    TP = int(cm[0, 0])
    FP = int(cm[0, 1])
    FN = int(cm[1, 0])
    TN = 0  # Tidak bermakna untuk object detection
    print(f"TP: {TP}, FP: {FP}, TN: {TN}, FN: {FN}")

    # precision = float(metrics.box.precision[0])
    # recall = float(metrics.box.recall[0])
    # f1 = 2 * precision * recall / (precision + recall + 1e-9)
    precision = float(metrics.box.p[0])
    recall = float(metrics.box.r[0])
    f1 = float(metrics.box.f1[0])


    mAP50 = float(metrics.box.map50)
    mAP5095 = float(metrics.box.map)

    rows.append([
        idx,
        TP, FP, TN, FN,
        precision, recall, f1,
        mAP50, mAP5095
    ])

# ===================== SIMPAN CSV =====================
with open(CSV_OUT, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(rows)

print("\n✅ Validation selesai")
print(f"Hasil disimpan ke: {CSV_OUT}")



Validating model iterasi 1
Ultralytics 8.3.221  Python-3.11.9 torch-2.9.0+cpu CPU (12th Gen Intel Core i5-1235U)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access  (ping: 0.10.0 ms, read: 410.379.4 MB/s, size: 362.3 KB)
val: Scanning C:\Users\USER\Documents\a skripsi\riset\data_eval_final\test generalisasi\test\labels.cache... 522 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 522/522 525.4Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 33/33 1.1it/s 30.5s0.9ss
                   all        522        522      0.992      0.987      0.992      0.846
Speed: 0.6ms preprocess, 41.0ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to C:\Users\USER\Documents\a skripsi\workv1\app\notebook_penelitian\generalisasi_eval\i10-e10\iterasi_1
TP: 515, FP: 4, TN: 0, FN: 7

Validating model iterasi 2
Ultralytics 8.3.221  Python-3.11.9 torch-2.9.0+cpu CPU

In [9]:
import pandas as pd
import matplotlib.pyplot as plt

# Tentukan path file CSV sesuai lokasi di komputer Anda
path_hasil = r'C:\Users\USER\Documents\a skripsi\workv1\app\notebook_penelitian\generalisasi_eval\i10-e10\results.csv'

# Membaca data dari CSV
try:
    df = pd.read_csv(path_hasil)
    print("Data berhasil dimuat.")
except FileNotFoundError:
    print(f"File tidak ditemukan di: {path_hasil}")
    # Jika menjalankan di lingkungan lokal, pastikan path sudah benar.
    # Untuk keperluan demonstrasi, kita gunakan nama file lokal jika path absolut gagal.
    df = pd.read_csv('results.csv') 

# --- Plot 1: Precision, Recall, dan F1-Score ---
plt.clf()
plt.plot(df['iteration'], df['precision'], marker='o', label='Precision', linewidth=2)
plt.plot(df['iteration'], df['recall'], marker='s', label='Recall', linewidth=2)
plt.plot(df['iteration'], df['f1_score'], marker='^', label='F1-Score', linewidth=2)

plt.title('Evaluasi Precision, Recall, dan F1-Score (5 Epoch per Iterasi)')
plt.xlabel('Iterasi')
plt.ylabel('Nilai Metrik')
plt.xticks(df['iteration'])
plt.ylim(0, 1.05)
plt.grid(True, linestyle='--', alpha=0.7)
plt.legend(loc='lower right')
# plt.savefig('metrics_5epoch_plot.png')
plt.savefig(os.path.join(OUTPUT_DIR, 'metrics_5epoch_plot.png'))
print("Grafik 'metrics_5epoch_plot.png' telah disimpan.")

# --- Plot 2: mAP50 dan mAP50-95 ---
plt.clf()
plt.plot(df['iteration'], df['mAP50'], marker='o', label='mAP50', color='red', linewidth=2)
plt.plot(df['iteration'], df['mAP50_95'], marker='s', label='mAP50-95', color='blue', linewidth=2)

plt.title('Evaluasi mAP50 dan mAP50-95 (5 Epoch per Iterasi)')
plt.xlabel('Iterasi')
plt.ylabel('Nilai mAP')
plt.xticks(df['iteration'])
plt.ylim(0, 1.05)
plt.grid(True, linestyle='--', alpha=0.7)
plt.legend(loc='lower right')
plt.savefig(os.path.join(OUTPUT_DIR, 'map_5epoch_plot.png'))
print("Grafik 'map_5epoch_plot.png' telah disimpan.")

Data berhasil dimuat.
Grafik 'metrics_5epoch_plot.png' telah disimpan.
Grafik 'map_5epoch_plot.png' telah disimpan.


In [17]:
import pandas as pd
import matplotlib.pyplot as plt

# Tentukan path file CSV
path_hasil = r'C:\Users\USER\Documents\a skripsi\workv1\app\notebook_penelitian\generalisasi_eval\i10-e10\results.csv'
FONT_SIZE = 20
OUTPUT_DIR = "./generalisasi_eval/i10-e10"
try:
    df = pd.read_csv(path_hasil)
    print("Data berhasil dimuat.")
except FileNotFoundError:
    print("File tidak ditemukan, pastikan path sudah benar.")
    # Fallback jika dijalankan di environment lain
    df = pd.read_csv('results.csv') 

# --- Pengaturan Font ---
# Mengatur font global (seperti Times New Roman jika tersedia, atau sans-serif)
plt.rcParams.update({'font.family': 'sans-serif', 'font.size': FONT_SIZE})
font_title = {'fontsize': FONT_SIZE, 'fontweight': 'bold'}
font_label = {'fontsize': FONT_SIZE}

# --- Membuat Subplot (1 Baris, 2 Kolom) ---
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Subplot Kiri: Precision, Recall, dan F1-Score
ax1.plot(df['iteration'], df['precision'], marker='o', label='Precision', linewidth=2)
ax1.plot(df['iteration'], df['recall'], marker='s', label='Recall', linewidth=2)
ax1.plot(df['iteration'], df['f1_score'], marker='^', label='F1-Score', linewidth=2)
ax1.set_title('Precision, Recall, dan F1-Score', fontdict=font_title, fontsize=FONT_SIZE)
ax1.set_xlabel('Iterasi', fontdict=font_label, fontsize=FONT_SIZE)
ax1.set_ylabel('Nilai Metrik', fontdict=font_label, fontsize=FONT_SIZE)
ax1.set_xticks(df['iteration'])
ax1.set_ylim(0, 1.05)
ax1.grid(True, linestyle='--', alpha=0.6)
ax1.legend(loc='lower right', fontsize=FONT_SIZE)

# Subplot Kanan: mAP50 dan mAP50-95
ax2.plot(df['iteration'], df['mAP50'], marker='o', label='mAP50', color='red', linewidth=2)
ax2.plot(df['iteration'], df['mAP50_95'], marker='s', label='mAP50-95', color='blue', linewidth=2)
ax2.set_title('mAP50 dan mAP50-95', fontdict=font_title, fontsize=FONT_SIZE)
ax2.set_xlabel('Iterasi', fontdict=font_label, fontsize=FONT_SIZE)
ax2.set_ylabel('Nilai mAP', fontdict=font_label, fontsize=FONT_SIZE)
ax2.set_xticks(df['iteration'])
ax2.set_ylim(0, 1.05)
ax2.grid(True, linestyle='--', alpha=0.6)
ax2.legend(loc='lower right', fontsize=FONT_SIZE)

# Menyesuaikan jarak antar subplot agar tidak tumpang tindih
plt.tight_layout()

# Simpan gambar
plt.savefig(os.path.join(OUTPUT_DIR, 'combined_evaluation_plot.png'), dpi=300)
print("Grafik gabungan 'combined_evaluation_plot.png' telah disimpan.")

Data berhasil dimuat.
Grafik gabungan 'combined_evaluation_plot.png' telah disimpan.


In [11]:
# Epoch 15
import os
from ultralytics import YOLO
import csv

# =========================
# KONFIGURASI UTAMA
# =========================

# Path data.yaml generalisasi
DATA_YAML = r"C:/Users/USER/Documents/a skripsi/riset/data_eval_final/test generalisasi/data.yaml"

# Output folder utama
OUTPUT_DIR = "./generalisasi_eval/i10-e15"

ROOT_DIR = r"D:\15-ep-10-it\15-ep-10-it"

# Daftar model (10 path)
MODEL_PATHS = [
    os.path.join(ROOT_DIR, f"1-iter/model1/weights/best.pt"),
    os.path.join(ROOT_DIR, f"2-iter/model2/weights/best.pt"),
    os.path.join(ROOT_DIR, f"3-iter/model3/weights/best.pt"),
    os.path.join(ROOT_DIR, f"4-iter/model4/weights/best.pt"),
    os.path.join(ROOT_DIR, f"5-iter/model5/weights/best.pt"),
    os.path.join(ROOT_DIR, f"6-iter/model6/weights/best.pt"),
    os.path.join(ROOT_DIR, f"7-iter/model7/weights/best.pt"),
    os.path.join(ROOT_DIR, f"8-iter/model8/weights/best.pt"),
    os.path.join(ROOT_DIR, f"9-iter/model9/weights/best.pt"),
    os.path.join(ROOT_DIR, f"10-iter/model10/weights/best.pt"),
]

CSV_OUT = os.path.join(OUTPUT_DIR, "results.csv")

os.makedirs(OUTPUT_DIR, exist_ok=True)

# ===================== HEADER CSV =====================
header = [
    "iteration",
    "TP", "FP", "TN", "FN",
    "precision", "recall", "f1_score",
    "mAP50", "mAP50_95"
]

rows = []

# ===================== LOOP VALIDASI =====================
for idx, model_path in enumerate(MODEL_PATHS, start=1):
    print(f"\nValidating model iterasi {idx}")
    model = YOLO(model_path)

    metrics = model.val(
        data=DATA_YAML,
        imgsz=640,
        conf=0.5,
        # iou=0.5,
        project=OUTPUT_DIR,
        name=f"iterasi_{idx}",
        save_json=False,
        plots=True
    )

    # Confusion Matrix
    cm = metrics.confusion_matrix.matrix
    # Format:
    # [[TP, FP],
    #  [FN, TN(background)]]

    TP = int(cm[0, 0])
    FP = int(cm[0, 1])
    FN = int(cm[1, 0])
    TN = 0  # Tidak bermakna untuk object detection
    print(f"TP: {TP}, FP: {FP}, TN: {TN}, FN: {FN}")

    # precision = float(metrics.box.precision[0])
    # recall = float(metrics.box.recall[0])
    # f1 = 2 * precision * recall / (precision + recall + 1e-9)
    precision = float(metrics.box.p[0])
    recall = float(metrics.box.r[0])
    f1 = float(metrics.box.f1[0])


    mAP50 = float(metrics.box.map50)
    mAP5095 = float(metrics.box.map)

    rows.append([
        idx,
        TP, FP, TN, FN,
        precision, recall, f1,
        mAP50, mAP5095
    ])

# ===================== SIMPAN CSV =====================
with open(CSV_OUT, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(rows)

print("\n✅ Validation selesai")
print(f"Hasil disimpan ke: {CSV_OUT}")



Validating model iterasi 1
Ultralytics 8.3.221  Python-3.11.9 torch-2.9.0+cpu CPU (12th Gen Intel Core i5-1235U)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access  (ping: 0.10.0 ms, read: 484.3454.3 MB/s, size: 355.2 KB)
val: Scanning C:\Users\USER\Documents\a skripsi\riset\data_eval_final\test generalisasi\test\labels.cache... 522 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 522/522  0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 33/33 0.5it/s 1:062.8sss
                   all        522        522      0.993      0.987      0.994      0.825
Speed: 1.2ms preprocess, 84.7ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to C:\Users\USER\Documents\a skripsi\workv1\app\notebook_penelitian\generalisasi_eval\i10-e15\iterasi_1
TP: 517, FP: 32, TN: 0, FN: 5

Validating model iterasi 2
Ultralytics 8.3.221  Python-3.11.9 torch-2.9.0+cpu CPU (12th G

In [12]:
import pandas as pd
import matplotlib.pyplot as plt

# Tentukan path file CSV sesuai lokasi di komputer Anda
path_hasil = r'C:\Users\USER\Documents\a skripsi\workv1\app\notebook_penelitian\generalisasi_eval\i10-e15\results.csv'

# Membaca data dari CSV
try:
    df = pd.read_csv(path_hasil)
    print("Data berhasil dimuat.")
except FileNotFoundError:
    print(f"File tidak ditemukan di: {path_hasil}")
    # Jika menjalankan di lingkungan lokal, pastikan path sudah benar.
    # Untuk keperluan demonstrasi, kita gunakan nama file lokal jika path absolut gagal.
    df = pd.read_csv('results.csv') 

# --- Plot 1: Precision, Recall, dan F1-Score ---
plt.clf()
plt.plot(df['iteration'], df['precision'], marker='o', label='Precision', linewidth=2)
plt.plot(df['iteration'], df['recall'], marker='s', label='Recall', linewidth=2)
plt.plot(df['iteration'], df['f1_score'], marker='^', label='F1-Score', linewidth=2)

plt.title('Evaluasi Precision, Recall, dan F1-Score (5 Epoch per Iterasi)')
plt.xlabel('Iterasi')
plt.ylabel('Nilai Metrik')
plt.xticks(df['iteration'])
plt.ylim(0, 1.05)
plt.grid(True, linestyle='--', alpha=0.7)
plt.legend(loc='lower right')
# plt.savefig('metrics_5epoch_plot.png')
plt.savefig(os.path.join(OUTPUT_DIR, 'metrics_5epoch_plot.png'))
print("Grafik 'metrics_5epoch_plot.png' telah disimpan.")

# --- Plot 2: mAP50 dan mAP50-95 ---
plt.clf()
plt.plot(df['iteration'], df['mAP50'], marker='o', label='mAP50', color='red', linewidth=2)
plt.plot(df['iteration'], df['mAP50_95'], marker='s', label='mAP50-95', color='blue', linewidth=2)

plt.title('Evaluasi mAP50 dan mAP50-95 (5 Epoch per Iterasi)')
plt.xlabel('Iterasi')
plt.ylabel('Nilai mAP')
plt.xticks(df['iteration'])
plt.ylim(0, 1.05)
plt.grid(True, linestyle='--', alpha=0.7)
plt.legend(loc='lower right')
plt.savefig(os.path.join(OUTPUT_DIR, 'map_5epoch_plot.png'))
print("Grafik 'map_5epoch_plot.png' telah disimpan.")

Data berhasil dimuat.
Grafik 'metrics_5epoch_plot.png' telah disimpan.
Grafik 'map_5epoch_plot.png' telah disimpan.


In [20]:
import pandas as pd
import matplotlib.pyplot as plt

# Tentukan path file CSV
path_hasil = r'C:\Users\USER\Documents\a skripsi\workv1\app\notebook_penelitian\generalisasi_eval\i10-e15\results.csv'
FONT_SIZE = 20
OUTPUT_DIR = "./generalisasi_eval/i10-e15"
try:
    df = pd.read_csv(path_hasil)
    print("Data berhasil dimuat.")
except FileNotFoundError:
    print("File tidak ditemukan, pastikan path sudah benar.")
    # Fallback jika dijalankan di environment lain
    df = pd.read_csv('results.csv') 

# --- Pengaturan Font ---
# Mengatur font global (seperti Times New Roman jika tersedia, atau sans-serif)
plt.rcParams.update({'font.family': 'sans-serif', 'font.size': FONT_SIZE})
font_title = {'fontsize': FONT_SIZE, 'fontweight': 'bold'}
font_label = {'fontsize': FONT_SIZE}

# --- Membuat Subplot (1 Baris, 2 Kolom) ---
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Subplot Kiri: Precision, Recall, dan F1-Score
ax1.plot(df['iteration'], df['precision'], marker='o', label='Precision', linewidth=2)
ax1.plot(df['iteration'], df['recall'], marker='s', label='Recall', linewidth=2)
ax1.plot(df['iteration'], df['f1_score'], marker='^', label='F1-Score', linewidth=2)
ax1.set_title('Precision, Recall, dan F1-Score', fontdict=font_title, fontsize=FONT_SIZE)
ax1.set_xlabel('Iterasi', fontdict=font_label, fontsize=FONT_SIZE)
ax1.set_ylabel('Nilai Metrik', fontdict=font_label, fontsize=FONT_SIZE)
ax1.set_xticks(df['iteration'])
ax1.set_ylim(0, 1.05)
ax1.grid(True, linestyle='--', alpha=0.6)
ax1.legend(loc='lower right', fontsize=FONT_SIZE)

# Subplot Kanan: mAP50 dan mAP50-95
ax2.plot(df['iteration'], df['mAP50'], marker='o', label='mAP50', color='red', linewidth=2)
ax2.plot(df['iteration'], df['mAP50_95'], marker='s', label='mAP50-95', color='blue', linewidth=2)
ax2.set_title('mAP50 dan mAP50-95', fontdict=font_title, fontsize=FONT_SIZE)
ax2.set_xlabel('Iterasi', fontdict=font_label, fontsize=FONT_SIZE)
ax2.set_ylabel('Nilai mAP', fontdict=font_label, fontsize=FONT_SIZE)
ax2.set_xticks(df['iteration'])
ax2.set_ylim(0, 1.05)
ax2.grid(True, linestyle='--', alpha=0.6)
ax2.legend(loc='lower right', fontsize=FONT_SIZE)

# Menyesuaikan jarak antar subplot agar tidak tumpang tindih
plt.tight_layout()

# Simpan gambar
plt.savefig(os.path.join(OUTPUT_DIR, 'combined_evaluation_plot.png'), dpi=300)
print("Grafik gabungan 'combined_evaluation_plot.png' telah disimpan.")

Data berhasil dimuat.
Grafik gabungan 'combined_evaluation_plot.png' telah disimpan.
